In [34]:
import sys, os
parent_dir = os.getcwd()
path = os.path.dirname(parent_dir)
sys.path.append(path)

from gym import spaces
from gym_homer.envs.test_env_v0 import HomerEnv
import pandas as pd
import numpy as np

In [35]:
data = pd.read_csv(path+"/test_env_data.csv", index_col=False).fillna(0)
env = HomerEnv(data=data)
pd.DataFrame(data)

,time_x,time_y,weekend,month_x,month_y,region_1,region_2,region_3,solar,loads,import_tariff,export_tariff,max_d,max_c,soc
0,0.000,1.000,0,0.0,1.0,0,0,0,0,1,0.05,0.0,0.0,0.0,0.0
1,0.500,0.866,0,0.0,1.0,0,0,0,-1,1,0.05,0.0,0.0,0.0,0.0
2,0.866,0.500,0,0.0,1.0,0,0,0,-1,1,0.05,0.0,0.0,0.0,0.0
3,1.000,0.000,0,0.0,1.0,0,0,0,0,1,0.05,0.0,0.0,0.0,0.0
4,0.866,-0.500,0,0.0,1.0,0,0,0,0,1,0.05,0.0,0.0,0.0,0.0
5,0.500,-0.866,0,0.0,1.0,0,0,0,0,1,0.05,0.0,0.0,0.0,0.0
6,0.000,-1.000,0,0.0,1.0,0,0,0,0,1,0.05,0.0,0.0,0.0,0.0
7,-0.500,-0.866,0,0.0,1.0,0,0,0,0,1,0.05,2.0,0.0,0.0,0.0
8,-0.866,-0.500,0,0.0,1.0,0,0,0,0,1,0.05,2.0,0.0,0.0,0.0
9,-1.000,-0.000,0,0.0,1.0,0,0,0,0,1,0.05,0.0,0.0,0.0,0.0


In [36]:
import gym, torch, numpy as np, torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import tianshou as ts
from gym import spaces, wrappers

In [37]:
#task = 'HomerEnv-v0'
lr, epoch, batch_size = 1e-3, 30, 64
train_num, test_num = 10, 100
gamma, n_step, target_freq = 0.9, 12, 320
buffer_size = 30000
eps_train, eps_test = 0.1, 0.05
step_per_epoch, step_per_collect = 10000, 15
logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))  # TensorBoard is supported!
# For other loggers: https://tianshou.readthedocs.io/en/master/tutorials/logger.html

In [38]:
# you can also try with SubprocVectorEnv
train_envs = ts.env.DummyVectorEnv([lambda: HomerEnv(data=data) for _ in range(train_num)])
test_envs = ts.env.DummyVectorEnv([lambda: HomerEnv(data=data) for _ in range(test_num)])

In [39]:
from tianshou.utils.net.common import Net
# you can define other net by following the API:
# https://tianshou.readthedocs.io/en/master/tutorials/dqn.html#build-the-network
#env = gym.make('gym_homer/HomerEnv-v0', data=data)
wrapped_env = wrappers.FlattenObservation(env)
state_shape = wrapped_env.observation_space.shape or wrapped_env.observation_space.n
action_shape = wrapped_env.action_space.shape or wrapped_env.action_space.n
net = Net(state_shape=state_shape, action_shape=action_shape, hidden_sizes=[128, 128, 128])
optim = torch.optim.Adam(net.parameters(), lr=lr)

In [40]:
policy = ts.policy.DQNPolicy(net, optim, gamma, n_step, target_update_freq=target_freq)
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(buffer_size, train_num), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)  # because DQN uses epsilon-greedy method

In [41]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector, epoch, step_per_epoch, step_per_collect,
    test_num, batch_size, update_per_step=1 / step_per_collect,
    train_fn=lambda epoch, env_step: policy.set_eps(eps_train),
    test_fn=lambda epoch, env_step: policy.set_eps(eps_test),
    stop_fn=lambda mean_rewards: mean_rewards >= 1,
    logger=logger)
print(f'Finished training! Use {result["duration"]}')

Epoch #1:   0%|          | 0/10000 [00:00<?, ?it/s]/Users/mbtalexander/Berkeley/w210/home-energy-optimizer/ts/lib/python3.9/site-packages/tianshou/data/collector.py:255: UserWarning: n_step=15 is not a multiple of #env (10), which may cause extra transitions collected into the buffer.
  warnings.warn(
Epoch #1: 10001it [00:01, 5757.06it/s, env_step=10000, len=11, loss=0.001, n/ep=0, n/st=20, rew=-0.08]                           


Epoch #1: test_reward: -0.040731 ± 0.038379, best_reward: -0.040731 ± 0.038379 in #1


Epoch #2: 10001it [00:01, 5598.20it/s, env_step=20000, len=11, loss=0.001, n/ep=0, n/st=20, rew=-0.05]                           


Epoch #2: test_reward: -0.020770 ± 0.039222, best_reward: -0.020770 ± 0.039222 in #2


Epoch #3: 10001it [00:01, 5489.05it/s, env_step=30000, len=11, loss=0.001, n/ep=0, n/st=20, rew=-0.03]                           


...

Epoch #28: test_reward: -0.019513 ± 0.034386, best_reward: -0.014270 ± 0.029852 in #26


Epoch #29: 10001it [00:01, 5836.07it/s, env_step=290000, len=11, loss=0.001, n/ep=0, n/st=20, rew=-0.07]                           


Epoch #29: test_reward: -0.018500 ± 0.035815, best_reward: -0.014270 ± 0.029852 in #26


Epoch #30: 10001it [00:01, 5916.06it/s, env_step=300000, len=11, loss=0.001, n/ep=0, n/st=20, rew=-0.05]                           


Epoch #30: test_reward: -0.029757 ± 0.055643, best_reward: -0.014270 ± 0.029852 in #26
Finished training! Use 52.34s


In [42]:
policy.eval()
policy.set_eps(eps_test)
collector = ts.data.Collector(policy, env, exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

/Users/mbtalexander/Berkeley/w210/home-energy-optimizer/ts/lib/python3.9/site-packages/tianshou/data/collector.py:68: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


Step Reward: 0.0
Action : [2, 2]
Step Reward: 0.0
Action : [2, 2, 2]
Step Reward: -0.05000000074505806
Action : [2, 2, 2, 2]
Step Reward: 0.0
Action : [2, 2, 2, 2, 1]
Step Reward: 0.0
Action : [2, 2, 2, 2, 1, 1]
Step Reward: 0.0
Action : [2, 2, 2, 2, 1, 1, 1]
Step Reward: 0.0
Action : [2, 2, 2, 2, 1, 1, 1, 1]
Step Reward: 0.0
Action : [2, 2, 2, 2, 1, 1, 1, 1, 1]
Step Reward: 0.0
Action : [2, 2, 2, 2, 1, 1, 1, 1, 1, 1]
Step Reward: 0.0
Action : [2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]
Step Reward: 0.0
Action : [2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]


{'n/ep': 1,
 'n/st': 11,
 'rews': array([-0.05]),
 'lens': array([11]),
 'idxs': array([0]),
 'rew': -0.05000000074505806,
 'len': 11.0,
 'rew_std': 0.0,
 'len_std': 0.0}

In [43]:
pd.DataFrame(env.history)b

,reward,net,action,bat_output
0,-0.05,1.0,2,0.0
1,0.00,0.0,2,0.0
2,0.00,0.0,2,0.0
3,-0.05,1.0,2,0.0
4,0.00,0.0,1,-1.0
5,0.00,0.0,1,-1.0
6,0.00,0.0,1,-1.0
7,0.00,0.0,1,-1.0
8,0.00,0.0,1,-1.0
9,0.00,0.0,1,-1.0


In [45]:
env.episode_reward

-0.05000000074505806

In [44]:
pd.DataFrame(env.data_arr)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000,1.000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.05,0.0,-1.0,0.0,1.000000
1,0.500,0.866,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,1.0,0.05,0.0,-1.0,0.0,1.000000
2,0.866,0.500,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,1.0,0.05,0.0,-1.0,0.0,1.000000
3,1.000,0.000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.05,0.0,-1.0,0.0,1.000000
4,0.866,-0.500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.05,0.0,-1.0,0.0,1.000000
5,0.500,-0.866,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.05,0.0,-1.0,1.0,0.894591
6,0.000,-1.000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.05,0.0,-1.0,1.0,0.789181
7,-0.500,-0.866,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.05,2.0,-1.0,1.0,0.683772
8,-0.866,-0.500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.05,2.0,-1.0,1.0,0.578363
9,-1.000,-0.000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.05,0.0,-1.0,1.0,0.472954
